In [1]:
import duckdb

In [2]:
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [ ]:
#con.execute('''drop table if exists produtos''')

In [4]:
# selecionando o registro mais recente, removendo duplicatas pela coluna NATBR
df = con.execute("""
                 select * from (
                 SELECT *, ROW_NUMBER() OVER (PARTITION BY NATBR ORDER BY data_ingestao DESC) as row FROM bronze_z0019
                 where data_ingestao >= '2025-12-15'
                 ) where row = 1
                """).fetchdf()
df.head(10)            

,NATBR,MAKTX,WERKS,MAINS,LABST,nome_arquivio,data_ingestao,row
0,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2025-12-17 22:58:49.621316,1
1,10004,SERRA,BT50,100,200,z0019_2.csv,2025-12-17 22:59:10.275180,1
2,10003,PREGO,BT10,100,60,z0019_2.csv,2025-12-17 22:59:10.275180,1
3,10002,MARTELO,BT50,100,1500,z0019_1.csv,2025-12-17 22:58:49.621316,1
4,10005,MACHADO,BT50,100,100,z0019_2.csv,2025-12-17 22:59:10.275180,1


In [5]:
# removendo colunas desnecessárias
df_final = df.drop(columns=['nome_arquivio', 'data_ingestao', 'row'])
df_final.head(10)

,NATBR,MAKTX,WERKS,MAINS,LABST
0,10001,PARAFUSO,BT10,100,100
1,10004,SERRA,BT50,100,200
2,10003,PREGO,BT10,100,60
3,10002,MARTELO,BT50,100,1500
4,10005,MACHADO,BT50,100,100


In [6]:
# renomeando as colunas 
df_final = df_final.rename(columns={"NATBR":'id'})
df_final = df_final.rename(columns={"MAKTX":'nm_produto'})
df_final = df_final.rename(columns={"WERKS":'id_categoria'})
df_final = df_final.rename(columns={"MAINS":'id_fornecedor'})
df_final = df_final.rename(columns={"LABST":'vl_preco'})
df_final.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10001,PARAFUSO,BT10,100,100
1,10004,SERRA,BT50,100,200
2,10003,PREGO,BT10,100,60
3,10002,MARTELO,BT50,100,1500
4,10005,MACHADO,BT50,100,100


In [7]:
df_final.dtypes

id               object
nm_produto       object
id_categoria     object
id_fornecedor    object
vl_preco         object
dtype: object

In [8]:
df2 = df_final.copy()
df2 = df2.astype({'id':int,
                  'nm_produto':str,
                  'id_categoria':str,
                  'id_fornecedor':int,
                  'vl_preco':float
                }
    )
df2.dtypes

id                 int64
nm_produto        object
id_categoria      object
id_fornecedor      int64
vl_preco         float64
dtype: object

In [9]:
con.execute('''
    CREATE TABLE IF NOT EXISTS produtos (
            id bigint,
            nm_produto text,
            id_categoria text,
            id_fornecedor bigint,
            vl_preco float)
''')


In [10]:
con.execute('''insert into produtos select * from df2''')

In [11]:
df_resultado = con.execute('SELECT * FROM produtos').fetchdf()
df_resultado.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10001,PARAFUSO,BT10,100,100.0
1,10004,SERRA,BT50,100,200.0
2,10003,PREGO,BT10,100,60.0
3,10002,MARTELO,BT50,100,1500.0
4,10005,MACHADO,BT50,100,100.0


In [12]:
con.close()